In [ ]:
import networkx as nx
import requests
import json
from pathlib import Path

ENDPOINT = 'http://localhost:8000/api/'
#ENDPOINT = 'http://domset.algorithm.engineering:8000/api/'

In [ ]:
def add_tag(name, description = None, style=None):
    data = {
        'name': name,
    }

    if description is not None:
        data['description'] = description

    if style is not None:
        data['style'] = style

    response = requests.post(ENDPOINT + 'tags/new', json=data)
    return response.json()


add_tag('pace19exact', 'Exact Instance of PACE 2019 / Vertex Cover', 1)
add_tag('pace20exact', 'Exact Instance of PACE 2020 / Treedepth', 1)
add_tag('pace21exact', 'Exact Instance of PACE 2020 / Cluster Editing', 1)
add_tag('pace22exact', 'Exact Instance of PACE 2020 / Directed Feedback Vertex Set', 1)
add_tag('pace23exact', 'Exact Instance of PACE 2020 / Twinwidth', 1)

add_tag('random', 'Random graph', 2)
add_tag('gnp', 'Random G(n,p) graph', 2)


In [ ]:
def to_dimacs(G):
    n = G.number_of_nodes()
    m = G.number_of_edges()
    lines = []
    lines.append(f"p ds {n} {m}")
    for u, v in G.edges():
        lines.append(f"{u + 1} {v + 1}")
    return "\n".join(lines)

for n in [10, 50, 100]:
    for avgdeg in [2, 5, 10, 15, 20, 50]:
        if avgdeg * 3 >= n:
            continue

        for _ in range(10):
            p = avgdeg / (n - 1)
            G = nx.gnp_random_graph(n, p)
            dimacs = to_dimacs(G)

            req = {
                "name": f"gnp",
                "description": f"Random G(n, p) graph with n={n} and p={p:.2E}",
                "data": dimacs,
                "submitted_by": "manpen",
                "tags": ["random", "gnp"],
            }
            
            url = ENDPOINT + 'instances/new'
            x = requests.post(url, json = req).json()
            print(x)



In [ ]:
descs = {
    'pace19': 'Exact Instance of PACE 2019 / Vertex Cover',
    'pace20': 'Exact Instance of PACE 2020 / Treedepth',
    'pace21': 'Exact Instance of PACE 2020 / Cluster Editing',
    'pace22': 'Exact Instance of PACE 2020 / Directed Feedback Vertex Set',
    'pace23': 'Exact Instance of PACE 2020 / Twinwidth',
}



for x in Path("graphs").glob("**/*.gr"):
    if x.parent.name not in descs: 
        continue


    with open(x) as f:
        data = f.read()

    req = {
        "name": x.stem,
        "description": descs[x.parent.name],
        "data": data,
        "submitted_by": "manpen",
        "tags": [str(x.parent.name) + "exact"],
    }
    
    try:
        url = ENDPOINT + 'instances/new'
        x = requests.post(url, json = req).json()
        print(x)
    except:
        pass